In [ ]:
import ee
ee.Authenticate()  # Esto abre una ventana del navegador para autenticar
ee.Initialize()
import geemap.core as geemap

/Users/ruthparajo/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
# Inicializa la API de Earth Engine (debes autenticarte si es la primera vez)
ee.Initialize()

# Definir un rectángulo con las coordenadas (xmin, ymin, xmax, ymax)
roi = ee.Geometry.Rectangle([-114.30, 38.90, -114.20, 38.95])

# Crea la colección de imágenes de Landsat 8, filtrando por fechas y el área de interés (roi)
dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
  .filterDate('2021-05-01', '2021-06-01') \
  .filterBounds(roi)

# Aplica los factores de escala necesarios
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

dataset = dataset.map(apply_scale_factors)

# Calcular el NDVI para cada imagen de la colección
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

# Aplicar la función de NDVI a la colección
ndvi_collection = dataset.map(calculate_ndvi)

# Definir la visualización para el NDVI
ndvi_visualization = {
    'bands': ['NDVI'],
    'min': 0.0,
    'max': 1.0,
    'palette': ['blue', 'white', 'green'],
}

# Crear el mapa y centrarlo en el área de interés
m = geemap.Map()
m.centerObject(roi, 10)

# Añadir la colección NDVI al mapa
m.addLayer(ndvi_collection.mean().clip(roi), ndvi_visualization, 'NDVI')

# Mostrar el mapa
m
